In [1]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.preprocessing_schoolid import SchoolYear

The preprocessing is handled by the SchoolYear class and its methods.

In [2]:
sy_1819 = SchoolYear('../../data/chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Profile_Information_SY1819.csv',
                     '../../data/chicago_data_portal_csv_files/.ipynb_checkpoints/Chicago_Public_Schools_-_School_Progress_Reports_SY1819-checkpoint.csv')

The original data set has 651 schools. 

In [3]:
sy_1819.merged_df.shape

(651, 276)

In [4]:
sy_1819.isolate_high_schools()
sy_1819.drop_no_gr_schools()
sy_1819.drop_no_student_schools()
sy_1819.make_percent_low_income()
sy_1819.drop_specialed_options().head()

,School_ID,Legacy_Unit_ID,Finance_ID,Short_Name_sp,Long_Name_sp,Primary_Category_sp,Is_High_School,Is_Middle_School,Is_Elementary_School,Is_Pre_School,Summary,Administrator_Title,Administrator,Secondary_Contact_Title,Secondary_Contact,Address_sp,City_sp,State_sp,Zip_sp,Phone_sp,Fax_sp,CPS_School_Profile_sp,Website_sp,Facebook,Twitter,Youtube,Pinterest,Attendance_Boundaries,Grades_Offered_All,Grades_Offered,Student_Count_Total,Student_Count_Low_Income,Student_Count_Special_Ed,Student_Count_English_Learners,Student_Count_Black,Student_Count_Hispanic,Student_Count_White,Student_Count_Asian,Student_Count_Native_American,Student_Count_Other_Ethnicity,Student_Count_Asian_Pacific_Islander,Student_Count_Multi,Student_Count_Hawaiian_Pacific_Islander,Student_Count_Ethnicity_Not_Available,Statistics_Description,Demographic_Description,Dress_Code,PreK_School_Day,Kindergarten_School_Day,School_Hours,Freshman_Start_End_Time,After_School_Hours,Earliest_Drop_Off_Time,Classroom_Languages,Bilingual_Services,Refugee_Services,Title_1_Eligible,PreSchool_Inclusive,Preschool_Instructional,Significantly_Modified,Hard_Of_Hearing,Visual_Impairments,Transportation_Bus,Transportation_El,Transportation_Metra,School_Latitude_sp,School_Longitude_sp,Average_ACT_School,Mean_ACT,College_Enrollment_Rate_School,College_Enrollment_Rate_Mean,Graduation_Rate_School,Graduation_Rate_Mean,Overall_Rating,Rating_Status,Rating_Statement,Classification_Description,School_Year,Third_Contact_Title,Third_Contact_Name,Fourth_Contact_Title,Fourth_Contact_Name,Fifth_Contact_Title,Fifth_Contact_Name,Sixth_Contact_Title,Sixth_Contact_Name,Seventh_Contact_Title,Seventh_Contact_Name,Network,Is_GoCPS_Participant,Is_GoCPS_PreK,Is_GoCPS_Elementary,Is_GoCPS_High_School,Open_For_Enrollment_Date,Closed_For_Enrollment_Date,Short_Name_pr,Long_Name_pr,School_Type,Primary_Category_pr,Address_pr,City_pr,State_pr,Zip_pr,Phone_pr,Fax_pr,CPS_School_Profile_pr,Website_pr,Progress_Report_Year,Blue_Ribbon_Award_Year,Excelerate_Award_Gold_Year,Spot_Light_Award_Year,Improvement_Award_Year,Excellence_Award_Year,Student_Growth_Rating,Student_Growth_Description,Growth_Reading_Grades_Tested_Pct_ES,Growth_Reading_Grades_Tested_Label_ES,Growth_Math_Grades_Tested_Pct_ES,Growth_Math_Grades_Tested_Label_ES,Student_Attainment_Rating,Student_Attainment_Description,Attainment_Reading_Pct_ES,Attainment_Reading_Lbl_ES,Attainment_Math_Pct_ES,Attainment_Math_Lbl_ES,Culture_Climate_Rating,Culture_Climate_Description,School_Survey_Student_Response_Rate_Pct,School_Survey_Student_Response_Rate_Avg_Pct,School_Survey_Teacher_Response_Rate_Pct,School_Survey_Teacher_Response_Rate_Avg_Pct,School_Survey_Parent_Response_Rate_Pct,School_Survey_Parent_Response_Rate_Avg_Pct,Healthy_School_Certification,Healthy_School_Certification_Description,Creative_School_Certification,Creative_School_Certification_Description,NWEA_Reading_Growth_Grade_3_Pct,NWEA_Reading_Growth_Grade_3_Lbl,NWEA_Reading_Growth_Grade_4_Pct,NWEA_Reading_Growth_Grade_4_Lbl,NWEA_Reading_Growth_Grade_5_Pct,NWEA_Reading_Growth_Grade_5_Lbl,NWEA_Reading_Growth_Grade_6_Pct,NWEA_Reading_Growth_Grade_6_Lbl,NWEA_Reading_Growth_Grade_7_Pct,NWEA_Reading_Growth_Grade_7_Lbl,NWEA_Reading_Growth_Grade_8_Pct,NWEA_Reading_Growth_Grade_8_Lbl,NWEA_Math_Growth_Grade_3_Pct,NWEA_Math_Growth_Grade_3_Lbl,NWEA_Math_Growth_Grade_4_Pct,NWEA_Math_Growth_Grade_4_Lbl,NWEA_Math_Growth_Grade_5_Pct,NWEA_Math_Growth_Grade_5_Lbl,NWEA_Math_Growth_Grade_6_Pct,NWEA_Math_Growth_Grade_6_Lbl,NWEA_Math_Growth_Grade_7_Pct,NWEA_Math_Growth_Grade_7_Lbl,NWEA_Math_Growth_Grade_8_Pct,NWEA_Math_Growth_Grade_8_Lbl,NWEA_Reading_Attainment_Grade_2_Pct,NWEA_Reading_Attainment_Grade_2_Lbl,NWEA_Reading_Attainment_Grade_3_Pct,NWEA_Reading_Attainment_Grade_3_Lbl,NWEA_Reading_Attainment_Grade_4_Pct,NWEA_Reading_Attainment_Grade_4_Lbl,NWEA_Reading_Attainment_Grade_5_Pct,NWEA_Reading_Attainment_Grade_5_Lbl,NWEA_Reading_Attainment_Grade_6_Pct,NWEA_Reading_Attainment_Grade_6_Lbl,NWEA_Reading_Attainment_Grade_7_Pct,NWEA_Readi

After dropping K-8 schools, schools with no graduation rates or student counts, and schools with non-traditional missions, the dataset is left with 124 schools. 

In [5]:
sy_1819.merged_df.shape

(124, 277)

# Train Test Split


For test size, I will isolate 20% of the data for test data. During cross validation, I will create 4 folds out of 100, so that each fold will have just about the same number of records as the holdout set.

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sy_1819.merged_df.drop("Graduation_Rate_School", axis =1),
                                                   sy_1819.merged_df["Graduation_Rate_School"], test_size=.2, 
                                                   random_state=42)

# FSM

The FSM only includes one predictor: `Student_Count_Total`, which has a high relative correlation to the target.  

In [7]:
df_train = X_train
df_train['Graduation_Rate_School'] = y_train


In [8]:
df_train[['Graduation_Rate_School', 'Student_Count_Total']].corr()

,Graduation_Rate_School,Student_Count_Total
Graduation_Rate_School,1.000000,0.456953
Student_Count_Total,0.456953,1.000000


In [9]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

cv_fsm = cross_validate(lr, X_train[['Student_Count_Total']], y_train, cv=4, return_train_score=True)
print(cv_fsm['train_score'].mean())
print(cv_fsm['test_score'].mean())

0.20981763588780716
0.1937363728503241


In [12]:
from src.cps_model import cv_cps

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
cv_cps(lr, X_train, y_train, features=['Student_Count_Total'])

Train Scores
[0.22718754 0.19027046 0.22837148 0.19344107]
0.20981763588780716
0.018001686997550177
##########################
Test Scores
[0.15100894 0.23404639 0.13353977 0.25635039]
0.1937363728503241
0.052427753215864216
##########################
Coefficients from fit on full train set
[0.00840968] 73.07458560082212


The first simple model is high bias.  Only ~21% of the variance is explained.  The test score R^2 have a comprable mean R2, but have a high standard deviation.



In this simple model, for every 8 students, for every 100 students, the graduation rate goes up .84 points.  

## Add Low Income Students

In [14]:
cv_cps(lr, X_train, y_train, ['Student_Count_Total', 'Student_Count_Low_Income'])

Train Scores
[0.23435008 0.23517025 0.23228612 0.19969077]
0.2253743040234015
0.014865582640729598
##########################
Test Scores
[ 0.14946568 -0.0601912   0.06646287  0.25915872]
0.10372401833853784
0.11673563913742296
##########################
Coefficients from fit on full train set
[ 0.01111313 -0.00447168] 73.71533750473652


Adding low income student count makes the test scores drop significantly.

## Add Percent Low Income Students

In [15]:
cv_cps(lr, X_train, y_train, ['Student_Count_Total', 'perc_low_income'])

Train Scores
[0.31134971 0.25147838 0.25045948 0.27776989]
0.27276436536736653
0.02482178990015859
##########################
Test Scores
[0.14259012 0.29270778 0.28110586 0.22045112]
0.2342137189879206
0.059591782115318376
##########################
Coefficients from fit on full train set
[ 5.98248587e-03 -2.33127917e+01] 94.62223005709221


Percent low income has a positive effect on both the training and test scores.  

One test set is performing significantly worse than the ohers at .14 R^2.

## One Hot Encode Networks

In [18]:
from sklearn.impute import SimpleImputer

network_imputer = SimpleImputer(strategy='constant', fill_value='missing')

X_train['Network'] = network_imputer.fit_transform(X_train[['Network']])

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder


# network_imputer = ColumnTransformer([('network_impute', SimpleImputer(strategy='constant', fill_value='missing'), ['Network'])], 
#                                          remainder='passthrough')


# network_transformer = ColumnTransformer( [('network_impute', SimpleImputer(strategy='constant', fill_value='missing'), ['Network']), 
#                                          ('network_ohe', OneHotEncoder(handle_unknown='ignore'), ['Network'])], 
#                                          remainder='passthrough')


network_ohe = ColumnTransformer([ 
                                          ('network_ohe', OneHotEncoder(handle_unknown='ignore'), ['Network'])], 
                                         remainder='passthrough')

network_pipe = make_pipeline(network_ohe, LinearRegression())

cv_cps(network_pipe, X_train, y_train, ['Student_Count_Total', 'perc_low_income', 'Network'])

Train Scores
[0.45440205 0.45415272 0.36770062 0.49544495]
0.442925083857444
0.04656940342440024
##########################
Test Scores
[ 0.30630564  0.28551318  0.50346845 -0.03052425]
0.2661907535277951
0.19126098814392287
##########################


Adding Network One Hot Encoded columns decreases the bias, but increases the vaiance.  There is even one negative R^2 score which is particularly worrisome.

# High Correlation 

In [ ]:
df_train.corr().iloc[-1].dropna().sort_values()

In [ ]:
df_train.corr().iloc[-1].dropna().sort_values(ascending=False)

In [ ]:
df_train[['Graduation_Rate_School', 'Student_Count_Low_Income']].corr()